# CoroBot Kinematics Solver for 6 DOF Manipulator

    This tutorial shows you how to simulate and create a controller for a 6 DOF robotic manipulator. The dimensions and model used in this iPython Notebook is based off of the standard manipulators used on the CoroBot models

# Problem Statement

The problem with kinematics and inverse kinematics is that as a robotic manipulator moves, describing each angle of the joints as well as the the positions of each joint. 

In [47]:
from IPython.display import Image

In [48]:
Image('insert path to image here')

# Theory of Kinematics

Insert stuff here

In [49]:
from __future__ import print_function, division #This is incase someone tries to run it in Python2
from sympy import symbols, simplify #CAS used for discussion of math contents
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point #Used for solving geometric  
from sympy.physics.vector import init_vprinting 
init_vprinting(use_latex='mathjax')

## Step One: Define our Reference Frames

In [50]:
Global_Frame = ReferenceFrame('Glob_Framethe base frame')
Base_Frame = ReferenceFrame('B_Frame')
Joint_One_Frame = ReferenceFrame('J1_Frame')
Joint_Two_Frame = ReferenceFrame('J2_Frame')
Joint_Three_Frame = ReferenceFrame('J3_Frame')
Joint_Four_Frame = ReferenceFrame('J4_Frame')
Wrist_Joint_Frame = ReferenceFrame('Wrist_Frame')
Gripper_Frame = ReferenceFrame('Gripper_Frame')

Now we need to specify how the frames are oriented with respect to each other. To do this we need to define the generalized coordinates $\theta_1$, $\theta_2$, $\theta_3$, $\theta_4$, $\theta_5$, $\theta_6$ 

In [51]:
thetab, theta1, theta2, theta3, theta4, theta5 = dynamicsymbols('theta_b, theta1, theta2, theta3, theta4, theta5')

Next we will define the orientation of each of the links relative to the inertial reference frame; or the global coordinate system, as they rotate through various arbitrary angle $\theta_n$

In the next cell we will set the "Base_Frame" position relative to the "Global_Frame" with some rotation "theta1" that rotates about the Z axis.

In [52]:
Base_Frame.orient(Global_Frame, 'Axis', (thetab,Global_Frame.z ))

Now we will calculate the transformation matrix; also called the direction cosine matrix (DCM) relative to the base frame. 

In [53]:
Base_Frame.dcm(Global_Frame)

⎡cos(θ_b)   sin(θ_b)  0⎤
⎢                      ⎥
⎢-sin(θ_b)  cos(θ_b)  0⎥
⎢                      ⎥
⎣    0         0      1⎦

In [54]:
Global_Frame.dcm(Base_Frame)

⎡cos(θ_b)  -sin(θ_b)  0⎤
⎢                      ⎥
⎢sin(θ_b)  cos(θ_b)   0⎥
⎢                      ⎥
⎣   0          0      1⎦

Now we can rotate the arm's base relative to the Global_Frame which is what we will use as the global coordinates

In [55]:
Image('path to second joint rotation')

Next we will calculate the rotational matrix of the first joint relative to the revolute base joint.

In [56]:
Joint_One_Frame.orient(Base_Frame,'Axis',(theta1, Base_Frame.x))

In [57]:
Joint_One_Frame.dcm(Base_Frame)

⎡1     0         0   ⎤
⎢                    ⎥
⎢0  cos(θ₁)   sin(θ₁)⎥
⎢                    ⎥
⎣0  -sin(θ₁)  cos(θ₁)⎦

In [58]:
Joint_Two_Frame.orient(Joint_One_Frame, 'Axis',(theta2, Joint_One_Frame.x))

In [59]:
Joint_Two_Frame.dcm(Base_Frame)

⎡1                  0                                   0                 ⎤
⎢                                                                         ⎥
⎢0  -sin(θ₁)⋅sin(θ₂) + cos(θ₁)⋅cos(θ₂)  sin(θ₁)⋅cos(θ₂) + sin(θ₂)⋅cos(θ₁) ⎥
⎢                                                                         ⎥
⎣0  -sin(θ₁)⋅cos(θ₂) - sin(θ₂)⋅cos(θ₁)  -sin(θ₁)⋅sin(θ₂) + cos(θ₁)⋅cos(θ₂)⎦

In [60]:
Joint_Three_Frame.orient(Joint_Two_Frame, 'Axis',(theta3, Joint_Two_Frame.x))

In [61]:
Joint_Three_Frame.dcm(Global_Frame)

⎡                                                cos(θ_b)                     
⎢                                                                             
⎢ -((-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))⋅cos(θ₁) - (sin(θ₂)⋅cos(θ₃) + sin(θ₃)⋅
⎢                                                                             
⎣-(-(-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))⋅sin(θ₁) + (-sin(θ₂)⋅cos(θ₃) - sin(θ₃)

                                                                            si
                                                                              
cos(θ₂))⋅sin(θ₁))⋅sin(θ_b)    ((-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))⋅cos(θ₁) - 
                                                                              
⋅cos(θ₂))⋅cos(θ₁))⋅sin(θ_b)  (-(-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))⋅sin(θ₁) + 

n(θ_b)                                                                        
                                                                              
(sin(θ₂)⋅cos(θ₃) + sin(θ₃)⋅cos(θ₂))⋅sin(θ₁))⋅cos(θ

In [62]:
Joint_Four_Frame.orient(Joint_Three_Frame, 'Axis',(theta4, Joint_Three_Frame.x))

In [63]:
Joint_Four_Frame.dcm(Global_Frame)

⎡                                                                             
⎢                                                                             
⎢ -(-((-sin(θ₃)⋅sin(θ₄) + cos(θ₃)⋅cos(θ₄))⋅sin(θ₂) + (sin(θ₃)⋅cos(θ₄) + sin(θ₄
⎢                                                                             
⎣-((-(-sin(θ₃)⋅sin(θ₄) + cos(θ₃)⋅cos(θ₄))⋅sin(θ₂) + (-sin(θ₃)⋅cos(θ₄) - sin(θ₄

                            cos(θ_b)                                          
                                                                              
)⋅cos(θ₃))⋅cos(θ₂))⋅sin(θ₁) + ((-sin(θ₃)⋅sin(θ₄) + cos(θ₃)⋅cos(θ₄))⋅cos(θ₂) - 
                                                                              
)⋅cos(θ₃))⋅cos(θ₂))⋅cos(θ₁) - ((-sin(θ₃)⋅sin(θ₄) + cos(θ₃)⋅cos(θ₄))⋅cos(θ₂) + 

                                                                              
                                                                              
(sin(θ₃)⋅cos(θ₄) + sin(θ₄)⋅cos(θ₃))⋅sin(θ₂))⋅cos(θ

In [65]:
Wrist_Joint_Frame.orient(Joint_Four_Frame, 'Axis', (theta5, Joint_Four_Frame.x))

In [66]:
Wrist_Joint_Frame.dcm(Global_Frame)

⎡                                                                             
⎢                                                                             
⎢   -((-((-sin(θ₄)⋅sin(θ₅) + cos(θ₄)⋅cos(θ₅))⋅sin(θ₃) + (sin(θ₄)⋅cos(θ₅) + sin
⎢                                                                             
⎣-(-((-(-sin(θ₄)⋅sin(θ₅) + cos(θ₄)⋅cos(θ₅))⋅sin(θ₃) + (-sin(θ₄)⋅cos(θ₅) - sin(

                                                                              
                                                                              
(θ₅)⋅cos(θ₄))⋅cos(θ₃))⋅sin(θ₂) + ((-sin(θ₄)⋅sin(θ₅) + cos(θ₄)⋅cos(θ₅))⋅cos(θ₃)
                                                                              
θ₅)⋅cos(θ₄))⋅cos(θ₃))⋅sin(θ₂) + ((-sin(θ₄)⋅sin(θ₅) + cos(θ₄)⋅cos(θ₅))⋅cos(θ₃) 

                                                                 cos(θ_b)     
                                                                              
 - (sin(θ₄)⋅cos(θ₅) + sin(θ₅)⋅cos(θ₄))⋅sin(θ₃))⋅co

In [67]:
simplify(Wrist_Joint_Frame.dcm(Global_Frame))

⎡              cos(θ_b)                               sin(θ_b)                
⎢                                                                             
⎢-sin(θ_b)⋅cos(θ₁ + θ₂ + θ₃ + θ₄ + θ₅)  cos(θ₁ + θ₂ + θ₃ + θ₄ + θ₅)⋅cos(θ_b)  
⎢                                                                             
⎣sin(θ₁ + θ₂ + θ₃ + θ₄ + θ₅)⋅sin(θ_b)   -sin(θ₁ + θ₂ + θ₃ + θ₄ + θ₅)⋅cos(θ_b) 

              0             ⎤
                            ⎥
 sin(θ₁ + θ₂ + θ₃ + θ₄ + θ₅)⎥
                            ⎥
 cos(θ₁ + θ₂ + θ₃ + θ₄ + θ₅)⎦

In [68]:
Gripper_Frame.orient(Wrist_Joint_Frame, 'Axis', (theta5, Wrist_Joint_Frame.y))

In [71]:
simplify(Gripper_Frame.dcm(Global_Frame))

⎡-sin(θ₁ + θ₂ + θ₃ + θ₄ + θ₅)⋅sin(θ₅)⋅sin(θ_b) + cos(θ₅)⋅cos(θ_b)  sin(θ₁ + θ₂
⎢                                                                             
⎢             -sin(θ_b)⋅cos(θ₁ + θ₂ + θ₃ + θ₄ + θ₅)                           
⎢                                                                             
⎣sin(θ₁ + θ₂ + θ₃ + θ₄ + θ₅)⋅sin(θ_b)⋅cos(θ₅) + sin(θ₅)⋅cos(θ_b)   -sin(θ₁ + θ

 + θ₃ + θ₄ + θ₅)⋅sin(θ₅)⋅cos(θ_b) + sin(θ_b)⋅cos(θ₅)   -sin(θ₅)⋅cos(θ₁ + θ₂ + 
                                                                              
   cos(θ₁ + θ₂ + θ₃ + θ₄ + θ₅)⋅cos(θ_b)                    sin(θ₁ + θ₂ + θ₃ + 
                                                                              
₂ + θ₃ + θ₄ + θ₅)⋅cos(θ₅)⋅cos(θ_b) + sin(θ₅)⋅sin(θ_b)  cos(θ₁ + θ₂ + θ₃ + θ₄ +

θ₃ + θ₄ + θ₅)⎤
             ⎥
θ₄ + θ₅)     ⎥
             ⎥
 θ₅)⋅cos(θ₅) ⎦